In [25]:
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [5]:
import os
import shutil

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])

In [8]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100.0%


In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=32, shuffle=True, num_workers=2)

In [73]:
len(train_loader.dataset)  / 32    # 1563

1562.5

In [75]:
len(train_loader.dataset) * .02

1000.0

In [14]:
image, label = train_data[0]

In [18]:
image.size()

torch.Size([3, 32, 32])

In [19]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [57]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        # (W - F + 2P) / S + 1
        # (32 - 5 + 2*0) / 1 + 1 = 
        self.conv1 = nn.Conv2d(3, 12, 5) # ---> (12, 28, 28)    # nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2) # ---> (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # ---> (14 - 5 + 2*0) / 1 + 1 = 10 (24, 10, 10) ---(maxpool)---> 24, 5, 5
        # then flatten (24, 5, 5) ---> 24*5*5
        
        
        # Dense layers
        self.fc1 = nn.Linear(24 * self.conv2.kernel_size[0] * self.conv2.kernel_size[0], 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, self.fc1.in_features)
        # x = torch.flatten(x, 1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
model = NeuralNet().to(device)
        

In [48]:
conv_temp = nn.Conv2d(12, 24, 5)
conv_temp.kernel_size[0], conv_temp.out_channels, conv_temp.in_channels

(5, 24, 12)

In [65]:
learning_rate = .001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=.9)

In [66]:
n_epochs = 30
total_steps = len(train_loader)

for epoch in range(n_epochs):
    print(f'Training epoch: {epoch}...')
    running_loss = .0
    
    for i, (images, labels) in enumerate(train_loader):
        
        # to gpu
        images = images.to(device)
        labels = labels.to(device)
        
        # fw pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
         # bw pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss =+ loss.item()
        
        if (i+1) % 500 == 0:
            print(f'epoch: {epoch+1}/{n_epochs}, step: {i}/{total_steps}, loss: {loss.item():.4f}')
    
    # print(f'loss: {loss.item()}')
       

Training epoch: 0...
epoch: 1/30, step: 499/1563, loss: 2.2837
epoch: 1/30, step: 999/1563, loss: 2.1373
epoch: 1/30, step: 1499/1563, loss: 2.0689
Training epoch: 1...
epoch: 2/30, step: 499/1563, loss: 1.8062
epoch: 2/30, step: 999/1563, loss: 1.5917
epoch: 2/30, step: 1499/1563, loss: 1.4374
Training epoch: 2...
epoch: 3/30, step: 499/1563, loss: 1.6567
epoch: 3/30, step: 999/1563, loss: 1.4938
epoch: 3/30, step: 1499/1563, loss: 1.4815
Training epoch: 3...
epoch: 4/30, step: 499/1563, loss: 1.2574
epoch: 4/30, step: 999/1563, loss: 1.5673
epoch: 4/30, step: 1499/1563, loss: 1.4884
Training epoch: 4...
epoch: 5/30, step: 499/1563, loss: 1.5164
epoch: 5/30, step: 999/1563, loss: 1.3871
epoch: 5/30, step: 1499/1563, loss: 1.1417
Training epoch: 5...
epoch: 6/30, step: 499/1563, loss: 1.7663
epoch: 6/30, step: 999/1563, loss: 1.3683
epoch: 6/30, step: 1499/1563, loss: 1.1353
Training epoch: 6...
epoch: 7/30, step: 499/1563, loss: 1.3893
epoch: 7/30, step: 999/1563, loss: 0.7371
epoch: 

In [86]:
# evaluate

correct = 0
total = 0
model.eval()

with torch.no_grad():
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 68.55%


# Using

In [122]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

In [120]:
image_paths = []

def get_images(path='.'):
    image_list = []
    for i in os.listdir(path):
        os.path.splitext(i)[1] == 'jpg'
        if (os.path.isfile(i) and os.path.splitext(i)[1] == '.jpg'): 
            
            image_list.append(i)
    return image_list
    


In [121]:
get_images('../data_CIFAR10')

[]

In [127]:
image_paths = get_images()
image_paths

['hand-drawn-transport-truck_23-2149166246.jpg',
 'watercolor-vector-deer-isolated-on-600nw-2496492153.jpg']

In [128]:
images = [load_image(img) for img in image_paths]
images

[tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
 
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]]),
 tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,

In [130]:
model.eval()

with torch.no_grad():
    for image in images:
        image = image.to(device)
        output = model(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {classes[predicted.item()]}')

Prediction: truck
Prediction: horse
